In [5]:
import matplotlib.pyplot as plt
from ipywidgets import widgets, DatePicker, TimePicker, SelectMultiple, Layout
import numpy as np
import pandas as pd
import datetime
from utils.FileBuffer import FileBuffer
%matplotlib inline


In [6]:
def get_data_btw(buffer, start, end):
    current_data = buffer.current_data
    
    return current_data[(current_data['time'] >= start) & (current_data['time'] <= end)]

In [7]:
buffer = FileBuffer("data/data_p6_e20.csv")
buffer.next()
data_frame = buffer.current_data
columns = buffer.get_columns()

In [8]:

def generate_plot(start_date, start_hour, end_date, end_hour, tags):
    start = datetime.datetime.combine(start_date, start_hour)
    end = datetime.datetime.combine(end_date, end_hour)
    tags = np.array(tags).flatten()
    data_filter = get_data_btw(buffer, start, end)
    
    fig, ax = plt.subplots(figsize=(20,10))
    x_values = data_filter["time"]
    for i in range(len(tags)):
        y_values = data_filter[tags[i]]
        line, = ax.plot(x_values, y_values, label=tags[i], lw = 3)
    
    plt.xticks(rotation='vertical')
    plt.legend()
    plt.show()

start_date = DatePicker(value=data_frame["time"].min().date(), description='Inicio: ')
start_hour = TimePicker(value=data_frame["time"].min().time(), description='Hora:')
end_date = DatePicker(value=data_frame["time"].max().date(), description='Fin: ')
end_hour = TimePicker(value=data_frame["time"].max().time(), description='Hora:')
tags = SelectMultiple( options=columns, value=[columns[0]],  description='Tags:')
tags.layout = Layout(height = '400px', width = '500px')
interactive = widgets.interactive_output(generate_plot, {'start_date': start_date, 'start_hour': start_hour,'end_date': end_date, 'end_hour': end_hour, 'tags': tags})
widgets.VBox([
    widgets.HBox([start_date, start_hour]),
    widgets.HBox([end_date, end_hour]),
    widgets.HBox([interactive,tags])
])
